In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import plotly.express as px
import polars as pl
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from cleaner import DataCleaner
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from tools import (
    logging,
    bins_generator,
    import_datasets,
    decode_clean,

)
import eda_movies as eda

In [ ]:
link = "clean_datasets/directors_movies.parquet"
df = import_datasets(link, "parquet")

In [ ]:
print(df.head().to_markdown())

In [ ]:
df[df.titre_str.str.contains("Terminator")]